In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
from collections import defaultdict
import string
import tensorflow as tf
import re
import os
import time
from tensorflow import keras
from tensorflow.keras.layers import Dense , Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

        


In [2]:
EncoderLen=50
DecoderLen=50
Batch_Size=32
Buffersize= Batch_Size*4


In [3]:
train_df= pd.read_csv("../input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv")


In [4]:
train_df.drop(['source'],axis=1,inplace=True)
mask= (train_df['english_sentence'].str.len()>20) &(train_df['english_sentence'].str.len()<200)
train_df=train_df.loc[mask]
train_df.head()


,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [5]:
pd.set_option('display.max_colwidth',None)
train_df.head()

,english_sentence,hindi_sentence
0,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [6]:
eng= train_df['english_sentence']
hind= train_df['hindi_sentence']
eng= eng.apply(lambda x: "<SOS> "+str(x)+" <EOS>")
hind=hind.apply(lambda x: "<SOS> "+x+" <EOS>")


In [7]:
filters= '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
token='<unk>'
eng_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters=filters,oov_token= token)
hind_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters=filters,oov_token=token)
eng_tokenizer.fit_on_texts(eng)
hind_tokenizer.fit_on_texts(hind)
inputs=eng_tokenizer.texts_to_sequences(eng)
targets=hind_tokenizer.texts_to_sequences(hind)

In [8]:
Encoder_Vocab = len(eng_tokenizer.word_index) + 1
Decoder_Vocab = len(hind_tokenizer.word_index) + 1
print(Encoder_Vocab, Decoder_Vocab)

56620 65824


In [9]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,maxlen=EncoderLen,padding='post',truncating='post')
targets= tf.keras.preprocessing.sequence.pad_sequences(targets,maxlen=DecoderLen,padding='post',truncating='post')
inputs=tf.cast(inputs, dtype= tf.int64)
targets= tf.cast(targets, dtype= tf.int64)


2022-06-24 08:42:27.191079: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
inputs

<tf.Tensor: shape=(107217, 50), dtype=int64, numpy=
array([[   2, 2945,   66, ...,    0,    0,    0],
       [   2, 1625,   63, ...,    0,    0,    0],
       [   2,   15, 2733, ...,    0,    0,    0],
       ...,
       [   2,   18,   14, ...,    0,    0,    0],
       [   2,   55, 2493, ...,    0,    0,    0],
       [   2, 3031,  125, ...,    0,    0,    0]])>

In [11]:
dataset= tf.data.Dataset.from_tensor_slices((inputs,targets)).shuffle(Buffersize).batch(Batch_Size)

In [12]:
def get_angles(position,i,d_model):
    angle_rates= 1/ np.power(10000, (2*(i//2)) / np.float32(d_model))
    return postion * angle_rates
def positional_encoding(position,d_model):
    angle_rads= get_angles(
    np.arange(position)[:,np.newaxis],
    np.arange(d_model)[np.newaxis,:],
    d_model)
    
    angle_rads[: , 0::2 ]= np.sin(angle_rads[:, 0::2 ])
    angle_rads[: , 1::2 ]=np.cos(angle_rads[: , 1::2 ])
    
    pos_encoding= angle_rads[np.newaxis, ...]
    
    return tf.cast(pos_encoding , dtype=tf.float32)
    

In [13]:
def create_padding_mask(seq):
    seq= tf.cast(tf.math.equal(seq,0), tf.float32)
    return seq[:, tf.newaxis,tf.newaxis, :]

def create_look_ahead_mask(size):
    mask= 1 - tf.linalg.band_part(tf.ones((size,size)),-1,0)
    return mask

In [14]:
def scaled_dot_product_attention(q,k,v,mask):
    matmul_qk= tf.matmul(q,k,transpose_b=True)
    
    dk = tf.cast(tf.shape(k)[-1],tf.float32)
    scaled_attention_logits= matmul_qk/ tf.math.sqrt(dk)
    
# if mask is not None:
 #   scaled_attention_logits += mask
    